##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
# This file was heavily changed by Nico Jahn

# Code starts here...

In [3]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [4]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [5]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np

In [6]:
# get all data
mnist = tf.keras.datasets.mnist

# split data and normalize the input data in a range between 0 and 1
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# convert labels from sparse to one hot encoded vectors
y_train = tf.one_hot(y_train,10)
y_test = tf.one_hot(y_test,10)

In [7]:
# define hyperparameter to search for
HP_LR = hp.HParam('Learning Rate', hp.IntInterval(-3,-2)) # is later used as log10 power
HP_EPOCHS = hp.HParam('Number of Epochs', hp.Discrete([1,3,5])) # separate models are evaluated after n epochs

# collect all hyperparameter
hparams = [HP_EPOCHS,HP_LR]

In [8]:
# define metrics for evaluation
METRIC_CCE = 'categorical crossentropy'
METRIC_RECALL = 'recall'
METRIC_PRECISION = 'precision'

# https://en.wikipedia.org/wiki/Precision_and_recall#Imbalanced_data
METRIC_ACCURACY = 'accuracy'
METRIC_BALANCED_ACCURACY = 'balanced accuracy'
METRIC_F1 = 'f1 score'
#METRIC_PPCR = 'predicted positive condition rate'

# collect all metrics
metrics = [hp.Metric(METRIC_CCE, display_name='Crossentropy'), \
                    hp.Metric(METRIC_RECALL, display_name='Recall'), \
                    hp.Metric(METRIC_PRECISION, display_name='Precision'), \
                    hp.Metric(METRIC_ACCURACY, display_name='Accuracy'), \
                    hp.Metric(METRIC_BALANCED_ACCURACY, display_name='Balanced Accuracy'), \
                    hp.Metric(METRIC_F1, display_name='F1 Score'), \
                    #hp.Metric(METRIC_PPCR, display_name='PPCR'), \
                    ]

In [9]:
# create a default file writer
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(hparams=hparams,metrics=metrics,)

In [10]:
def train_test_model(hparams):
    # load your hyperparameter from the dict
    learning_rate = hparams[HP_LR]
    num_epochs = hparams[HP_EPOCHS]
    
    # creating the model
    model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(32, activation=tf.nn.relu),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10, activation=tf.nn.softmax),
    ])
    
    # compile/define optimizer and metrics
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=[   tf.keras.metrics.CategoricalCrossentropy(name='cce'), \
                        tf.keras.metrics.Recall(name='recall'), \
                        tf.keras.metrics.Precision(name='precision'), \
                        tf.keras.metrics.TruePositives(name='tp'), \
                        tf.keras.metrics.TrueNegatives(name='tn'), \
                        tf.keras.metrics.FalsePositives(name='fp'), \
                        tf.keras.metrics.FalseNegatives(name='fn')],
    )
    
    # train and evaluate on metrics
    model.fit(x_train, y_train, epochs=num_epochs)
    loss, cce, recall, precision, tp, tn, fp, fn = model.evaluate(x_test, y_test)
    
    # return necessary metrics
    return cce, recall, precision, tp, tn, fp, fn

In [11]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        step = hparams[HP_EPOCHS]
        
        # do the training procedure once
        # TODO: create n repetitions to create a mean value or do n-folds
        cce, recall, precision, tp, tn, fp, fn = train_test_model(hparams)
        
        # basic metrics which are precomputed for us
        tf.summary.scalar(METRIC_CCE, cce, step=step)
        tf.summary.scalar(METRIC_RECALL, recall, step=step)
        tf.summary.scalar(METRIC_PRECISION, precision, step=step)
        
        # more advanced metrics which we have to compute ourselves
        ## Accuracy = (tp+tn)/(tp+tn+fp+fn)
        tf.summary.scalar(METRIC_ACCURACY, (tp+tn)/(tp+tn+fp+fn), step=step)
        ## Balanced accuracy = (TPR+TNR)/2
        tf.summary.scalar(METRIC_BALANCED_ACCURACY, ((tp/(tp+fn))+(tn/(tn+fp)))/2, step=step)
        ## Predicted positive condition rate = (tp+fp)(tp+fp+tn+fn)
        #tf.summary.scalar(METRIC_PPCR, (tp+fp)/(tp+tn+fp+fn), step=step)
        ## F1 = 2* (precision*recall)/(precision+recall)
        tf.summary.scalar(METRIC_F1, 2*(precision*recall)/(precision+recall), step=step)

In [12]:
session_num = 0

for epochs in HP_EPOCHS.domain.values:
    min_, max_= HP_LR.domain.min_value, HP_LR.domain.max_value
    for lr in np.logspace(min_,max_,max_-min_+1):
        # write your hyperparameter to the dict
        hparams = {
                HP_EPOCHS: epochs,
                HP_LR: lr,
        }
        
        # 1 hp combination test
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)
        print()
        session_num += 1

--- Starting trial: run-0
{'Number of Epochs': 1, 'Learning Rate': 0.001}
313/313 [==============================] - 2s 6ms/step - loss: 0.2186 - cce: 0.2186 - recall: 0.9219 - precision: 0.9526 - tp: 9219.0000 - tn: 89541.0000 - fp: 459.0000 - fn: 781.0000

--- Starting trial: run-1
{'Number of Epochs': 1, 'Learning Rate': 0.01}
313/313 [==============================] - 2s 6ms/step - loss: 0.2186 - cce: 0.2186 - recall: 0.9292 - precision: 0.9458 - tp: 9292.0000 - tn: 89468.0000 - fp: 532.0000 - fn: 708.0000

--- Starting trial: run-2
{'Number of Epochs': 3, 'Learning Rate': 0.001}
Epoch 1/3
1875/1875 [==============================] - 11s 6ms/step - loss: 0.4780 - cce: 0.4780 - recall: 0.7955 - precision: 0.9218 - tp: 47731.0000 - tn: 535952.0000 - fp: 4048.0000 - fn: 12269.0000
Epoch 2/3
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2770 - cce: 0.2770 - recall: 0.8991 - precision: 0.9394 - tp: 53944.0000 - tn: 536520.0000 - fp: 3480.0000 - fn: 6056.0000
Epoch 

In [13]:
# Copied from: https://github.com/tensorflow/tensorboard/issues/3032#issuecomment-566140412
# NOTE: When starting Tensorboard via magic command, then i can't kill it properly in a container
# Therefore, couple it with the kernel and just restart the kernel later on
# This is a continous while loop, so this will stop code execution after it 
import tensorboard
import os
import time

tb = tensorboard.program.TensorBoard()
tb.configure(bind_all=True, logdir="logs/", port=6006)
url = tb.launch()
print("TensorBoard %s started at %s" % (tensorboard.__version__, url))
pid = os.getpid()
print("PID = %d; use 'kill %d' to quit" % (pid, pid))
while True:
    try:
        time.sleep(60)
    except KeyboardInterrupt:
        break
print()
print("Shutting down")

TensorBoard 2.2.1 started at http://b86465c257f6:6006/
PID = 5693; use 'kill 5693' to quit

Shutting down
